#### Run GeoTopic Parser End-to-End
1. Kill all java processes and kill tika server is already running (refer to Errors section fo ReadMe)
    - `killall java`
    - List current processes: `jps`
        - Kill all tika processes with: `kill <process number>`
2. Navigate to directory /3_Tika_GeoTopic_Parser and run command to start lucene server: `lucene-geo-gazetteer -server`
    - Should see this if working: `INFO: Starting ProtocolHandler ["http-nio-8765"]`
    - If its not working, re-run command to set up path inside of /lucene-geo-gazetteer/src/main/bin: `export PATH=$PWD:$PATH`
3. In new terminal window, navigate to directory /3_Tika_GeoTopic_Parser and run command to start geotopic server: `./geotopic-server`
    - Should see this if working: `INFO  [main] 16:25:04,222 org.apache.tika.server.core.TikaServerProcess Started Apache Tika server ff835cb6-9aa1-4817-ba8d-d035eb174c87 at http://localhost:9998/`
4. In new terminal window again, navigate to directory /3_Tika_GeoTopic_Parser and run command to test servers: `curl -T polar.geot -H "Content-Type: application/geotopic; filename=polar.geot" http://localhost:9998/rmeta | python -mjson.tool`
    - Should Get if running correctly: `United States, 39.76, -98.5`

In [2]:
from tika import parser
import os
import pandas as pd

In [3]:
df_master_dataset = pd.read_csv('../Datasets/Master_Dataset.tsv', delimiter='\t')

/var/folders/fn/pdhscw6s1wz7r1wk9ssprtz80000gn/T/ipykernel_18317/2785656331.py:1: DtypeWarning: Columns (0,2,5,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_master_dataset = pd.read_csv('../Datasets/Master_Dataset.tsv', delimiter='\t')


In [4]:
df_master_dataset['GeoTopic Name'] = pd.Series(dtype=str)
df_master_dataset['GeoTopic Latitude'] = pd.Series(dtype=float)
df_master_dataset['GeoTopic Longitude'] = pd.Series(dtype=float)

In [12]:
directory_path_v2 = 'Pixstory_Posts_Text_Files/'

directory = os.listdir(directory_path_v2)

In [14]:
# Takes about XX mins to run on Macbook Air 2020 M1 Chip
cnt = 0

for file in directory:

    cnt +=1

    # check if the current file is a file (not a subdirectory)
    # if os.path.isfile(os.path.join(directory_path_v2, file)):

    # split the string by '_' character and get the second element (index 1)
    index = int(file.split('_')[1])
    # print(index)

    # Parse the file with Tika and the GeoTopic Parser
    parsed = parser.from_file(directory_path_v2 + file, headers={'Content-Type' : 'application/geotopic'})

    try:
        name = parsed['metadata']['Geographic_NAME']
        latitude = parsed['metadata']['Geographic_LATITUDE']
        longitude = parsed['metadata']['Geographic_LONGITUDE']

        df_master_dataset.loc[index, 'GeoTopic Name'] = name
        df_master_dataset.loc[index, 'GeoTopic Latitude'] = latitude
        df_master_dataset.loc[index, 'GeoTopic Longitude'] = longitude

    except:
        df_master_dataset.loc[index, 'GeoTopic Name'] = "NaN"
        df_master_dataset.loc[index, 'GeoTopic Latitude'] = "NaN"
        df_master_dataset.loc[index, 'GeoTopic Longitude'] = "NaN"

    finally:
        print(cnt)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
df_master_dataset.head()

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,...,New Vaccinations,YouTube Trending Video Title,YouTube Trending Video Channel,YouTube Trending Video Category,YouTube Trending Video Views,YouTube Trending Video Likes,is_sarcastic,GeoTopic Name,GeoTopic Latitude,GeoTopic Longitude
0,121169.0,STY1659426957,103.0,USR1606807023,female,34.0,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,Barbie,35.0065,-6.07477
1,127727.0,STY1660634861,103.0,USR1606807023,female,34.0,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
2,123665.0,STY1660027898,103.0,USR1606807023,female,34.0,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
3,130517.0,STY1661151635,103.0,USR1606807023,female,34.0,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
4,125466.0,STY1660372361,109.0,USR1606851217,others,31.0,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2020-02-12 01:03:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,United States,39.76,-98.5


/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_2/GitHub/DSCI550-PixstoryMediaExtractionAndAnalysis/3_Tika_GeoTopic_Parser/Pixstory_Posts_Text_Files_TEST/post_633_126941.0.txt

/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_2/GitHub/DSCI550-PixstoryMediaExtractionAndAnalysis/3_Tika_GeoTopic_Parser/post_633_126941.0.txt

In [16]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,...,New Vaccinations,YouTube Trending Video Title,YouTube Trending Video Channel,YouTube Trending Video Category,YouTube Trending Video Views,YouTube Trending Video Likes,is_sarcastic,GeoTopic Name,GeoTopic Latitude,GeoTopic Longitude
0,121169.0,STY1659426957,103.0,USR1606807023,female,34.0,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,Barbie,35.0065,-6.07477
1,127727.0,STY1660634861,103.0,USR1606807023,female,34.0,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
2,123665.0,STY1660027898,103.0,USR1606807023,female,34.0,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
3,130517.0,STY1661151635,103.0,USR1606807023,female,34.0,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,2020-01-12 12:47:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,NaN,NaN,NaN
4,125466.0,STY1660372361,109.0,USR1606851217,others,31.0,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2020-02-12 01:03:00,...,0.0,I'm Coming Out,NikkeTutorials,Fashion,36400000.0,3000000.0,1.0,United States,39.76,-98.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94997,125349.0,STY1660325841,71665.0,USR1660325291,NaN,18.0,COLLEGE LIFE,MODR CLASS,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 22:58:00,...,46212.0,"100 Kids Vs 100 Adults For $500,000",MrBeast,Entertainment,50129360.0,2164163.0,1.0,NaN,NaN,NaN
94998,125345.0,STY1660325638,71673.0,USR1660325486,NaN,17.0,The one with the bestie😍,College life without a friend is useless. Enjo...,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:01:00,...,46212.0,"100 Kids Vs 100 Adults For $500,000",MrBeast,Entertainment,50129360.0,2164163.0,1.0,NaN,NaN,NaN
94999,125351.0,STY1660326276,71698.0,USR1660326026,NaN,21.0,A tale of friendship,Check this out.,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,...,46212.0,"100 Kids Vs 100 Adults For $500,000",MrBeast,Entertainment,50129360.0,2164163.0,1.0,NaN,NaN,NaN
95000,125355.0,STY1660326971,71697.0,USR1660326022,NaN,19.0,College,"Sheridan college, HazelMcCallion Canpus",https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,...,46212.0,"100 Kids Vs 100 Adults For $500,000",MrBeast,Entertainment,50129360.0,2164163.0,1.0,NaN,NaN,NaN


In [20]:
df_master_dataset.head(20)[["Narrative", "GeoTopic Name", "GeoTopic Latitude", "GeoTopic Longitude"]]

,Narrative,GeoTopic Name,GeoTopic Latitude,GeoTopic Longitude
0,"The colour of the year is here, and it's *drum...",Barbie,35.0065,-6.07477
1,We Indians do love to bastardise our foods- Ch...,NaN,NaN,NaN
2,Can professors not have personal lives? \n\nAd...,NaN,NaN,NaN
3,"She was recorded on video manhandling him, sho...",NaN,NaN,NaN
4,The House passed the Inflation Reduction Act o...,United States,39.76,-98.5
5,What's yours Ragi Gupta ? @Prakhar Dixit,NaN,NaN,NaN
6,How bad are things at ManU?\nBad time for to b...,NaN,NaN,NaN
7,The noose is tightening.\nThe former president...,Florida,28.75054,-82.5001
8,"Hi, whats up",NaN,NaN,NaN
9,You can never go wrong with Led Zep,NaN,NaN,NaN


In [17]:
# df_master_dataset.to_csv('../Datasets/Master_Dataset_GeoTopic_v2.csv', index=False)